In [1]:
%pip install pandas     
%pip install numpy
%pip install opencv-python-headless
%pip install protobuf
%pip install tensorflow
%pip install keras
%pip install scikit-learn
%pip install labelImg

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
"""
    Standard Python Machine Learning Libraries, for array structure and dataset loading
"""
import pandas as pd
import numpy as np


"""
    MediaPipe - used for handlandmark detection
    OpenCv - Video Image Data Extraction
    JSON | Protobuf - Data Formating
"""
import cv2
import json
from google.protobuf.json_format import MessageToJson


"""
    Libraries for file name processing and preparation
"""
import random
import string
import shutil
import os


"""
    Libraries to keep sequence consistency
"""
from keras.preprocessing.sequence import pad_sequences
import math

"""
    Libraries for splitting training and testing data and also transform our label classes
"""

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split


"""
    Neural Network Libraries
"""

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, BatchNormalization, Activation
from keras.callbacks import LearningRateScheduler

In [3]:
"""
    Utility Function for UID generation and renaming of files
"""

def generate_random_uid():
    """Generate a random UID."""
    return ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))



"""
    Remake files for class labeling and converting videos to frames
"""
def rename_videos(source_directory, destination_directory, class_names, idx):
    """Rename videos in the specified directory."""
    for root, dirs, files in os.walk(source_directory):
        for file in files:
            if file.endswith(".MOV"):
                old_path = os.path.join(root, file)

                # Convert .MOV to frames
                vidcap = cv2.VideoCapture(old_path)

                # Get the frame rate and calculate the number of frames to skip
                fps = vidcap.get(cv2.CAP_PROP_FPS)
                skip_frames_start = int(2 * fps)

                # Get the total number of frames and calculate the duration of the video in seconds
                total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
                video_duration = total_frames / fps

                # Calculate the timestamp to stop (2 seconds before the end)
                stop_timestamp = video_duration - 1.9

                # Convert the timestamp to frame index
                stop_frame_index = int(stop_timestamp * fps)

                success, image = vidcap.read()
                count = 0
                saved_frames_count = 0
                while success and count < stop_frame_index:
                    if count >= skip_frames_start:
                        frame_name = os.path.join(destination_directory, f"{class_names[idx]}", f"{generate_random_uid()}_{saved_frames_count}.jpg")
                        os.makedirs(os.path.dirname(frame_name), exist_ok=True)
                        cv2.imwrite(frame_name, image)     # save frame as JPEG file
                        saved_frames_count += 1
                    success, image = vidcap.read()
                    count += 1

                print(f"Converted {file} to {saved_frames_count} frames")
                
""" 
    Update Target Classes Filenames and move them to a single directory for dataset processing
"""
# Specify the source directory containing the videos
source_directory = './dataset/clips/'

# Specify the destination directory for the renamed videos
destination_directory = './dataset/od_data/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_directory, exist_ok=True)

# Call the function to rename and copy the videos
targetSets = [85, 86, 89, 94]

# Specify the class names for each index
class_names = ['bread', 'egg', 'chicken', 'crab']

for idx, target in enumerate(targetSets):
    print("Renaming Target Set: ", target)
    rename_videos(source_directory + str(target), destination_directory, class_names, idx)
    print("\n\n")


Renaming Target Set:  85
Converted 11taoceh_0.MOV to 10 frames
Converted 4asdlwz1_0.MOV to 10 frames
Converted 4wjsa5wq_0.MOV to 11 frames
Converted 940bqr4z_0.MOV to 10 frames
Converted 9dppbec3_0.MOV to 11 frames
Converted f9r9bsva_0.MOV to 10 frames
Converted fdpnld0t_0.MOV to 10 frames
Converted fx5jgfby_0.MOV to 11 frames
Converted gwa534to_0.MOV to 10 frames
Converted i6641z1q_0.MOV to 11 frames
Converted inl5n07c_0.MOV to 11 frames
Converted mato6gvs_0.MOV to 10 frames
Converted nz7q913t_0.MOV to 10 frames
Converted opunh4zs_0.MOV to 10 frames
Converted p5ga19wa_0.MOV to 10 frames
Converted qygo7g5s_0.MOV to 10 frames
Converted r4znhltx_0.MOV to 11 frames
Converted titl2dvs_0.MOV to 10 frames
Converted tzd3o8eu_0.MOV to 10 frames
Converted vvzajti5_0.MOV to 10 frames



Renaming Target Set:  86
Converted 0.MOV to 10 frames
Converted 1.MOV to 10 frames
Converted 10.MOV to 10 frames
Converted 11.MOV to 10 frames
Converted 12.MOV to 10 frames
Converted 13.MOV to 10 frames
Converted